In [ ]:
import json
import os
import pickle
from datetime import datetime
import threading
import shap
import torch
import numpy as np
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer, util
import faiss

# ----------------------------
# Jouw code (inline gebracht)
# ----------------------------

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Llama-3.2-3B-Instruct-Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048, verbose=False)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages.json', 
                 knowledge_file='knowledge.json', 
                 faiss_index_file='faiss_index.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def search_knowledge(self, query, top_k=5):
        if self.index is None or len(self.knowledge_data) == 0:
            return []
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)
        results = []
        for idx in indices[0]:
            if idx == -1:
                continue
            results.append(self.knowledge_data[idx])
        return results

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

# ----------------------------
# SHAP-explainer
# ----------------------------

chatbot = Chatbot()
llm = chatbot.llm
scorer_model = SentenceTransformer("all-MiniLM-L6-v2")

vraag = "Waarom wordt er een silincer geplaatst?"
doelantwoord = "Om het geluidsniveau van de luchtuitlaat te verminderen."

matches = chatbot.search_knowledge(vraag, top_k=5)
fragmenten = [f"{t['subject']} {t['predicate']} {t['object']}" for t in matches]

def bouw_prompt(fragment_string):
    current_time = datetime.utcnow().isoformat()
    system_message = (
        "You are a helpful assistent. Your task is to answer questions. "
        "Think step by step. I'm going to tip $300K for a better solution! "
        "You will be penalized for incorrect answers. Answer in Dutch.; "
        f"Current date and time: {current_time}\n"
        "Answer based on retrieved knowledge:\n"
        f"{fragment_string}\n"
        "Answer the questions based only on the retrieved knowledge!"
    )
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": vraag}
    ]

def antwoord_score(output):
    emb_out = scorer_model.encode(output, convert_to_tensor=True)
    emb_ref = scorer_model.encode(doelantwoord, convert_to_tensor=True)
    return util.pytorch_cos_sim(emb_out, emb_ref).item()

def shap_predict(inputs):
    scores = []
    for masked_fragments_str in inputs:
        prompt = bouw_prompt(masked_fragments_str)
        try:
            response = llm.create_chat_completion(
                messages=prompt,
                temperature=0.0,
                max_tokens=200
            )['choices'][0]['message']['content']
        except Exception:
            response = ""
        score = antwoord_score(response)
        scores.append(score)
    return scores

# SHAP inputdata opbouwen
data = ["\n".join([f"- {frag}" for frag in fragmenten])]

explainer = shap.Explainer(shap_predict, shap.maskers.Text(tokenizer=None))
shap_values = explainer(data)

shap.plots.text(shap_values)


## Concreet voorbeeld

In [ ]:
import json
import os
import pickle
from datetime import datetime
from llama_cpp import Llama
import threading
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Llama-3.2-3B-Instruct-Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048, verbose=False)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages_vidar_stabilizer_pressure_control_2.json', 
                 knowledge_file='knowledge_vidar.json', 
                 faiss_index_file='faiss_index_vidar.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def save_message(self, role, content):
        messages = self.load_json_data(self.messages_file)
        message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
        messages.append(message)
        self.save_json_data(self.messages_file, messages)

    def save_knowledge(self, triplets):
        if not triplets:
            return
        knowledge = self.load_json_data(self.knowledge_file)
        existing_set = {(t['subject'], t['predicate'], t['object']) for t in knowledge}
        new_triplets = []
        for triplet in triplets:
            triplet['timestamp'] = datetime.utcnow().isoformat()
            key = (triplet['subject'], triplet['predicate'], triplet['object'])
            if key not in existing_set:
                knowledge.append(triplet)
                new_triplets.append(triplet)
                existing_set.add(key)
        self.save_json_data(self.knowledge_file, knowledge)
        if new_triplets:
            self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

    def search_knowledge(self, query, top_k=5):
        if self.index is None or len(self.knowledge_data) == 0:
            return []
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)
        results = []
        for idx in indices[0]:
            if idx == -1:
                continue
            results.append(self.knowledge_data[idx])
        return results

    def generate_response(self, conversation_history, user_message):
        knowledge_matches = self.search_knowledge(user_message, top_k=5)
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_matches:
            system_message += "Answer based on retrieved knowledge:\n"
            for t in knowledge_matches:
                system_message += f"- {t['subject']} {t['predicate']} {t['object']} (Videotimestamps: start: {t['start']}, end: {t['end']})\n"
                system_message += "Answer the questions based only on the retrieved knowledge! \n"
        else:
            system_message += "No direct related knowledge found. Proceeding with general reasoning.\n"

        enriched_history = [{"role": "system", "content": f"You are a helpful assistent. Your task is to answer questions. Think step by step. I'm going to tip $300K for a better solution! You will be penalized for incorrect answers. Answer in Dutch.; {system_message}"}]
        enriched_history.append({"role": "user", "content": user_message})

        response = self.llm.create_chat_completion(
            messages=enriched_history,
            temperature=0.7,
        )['choices'][0]['message']['content']

        eval_entry = {
            "query": user_message,
            "contexts": [f"{t['subject']} {t['predicate']} {t['object']}" for t in knowledge_matches],
            "answer": response,
        }
        with open("evaluation_logs.jsonl", "a", encoding="utf-8") as f:
            f.write(json.dumps(eval_entry) + "\n")

        return response

    def chat(self, questions=None):
        if questions:
            print("Chatbot is bezig met batch vragen beantwoorden.")
            for user_message in questions:
                print(f"You: {user_message}")
                self.save_message(role='user', content=user_message)
                conversation = self.load_json_data(self.messages_file)[-3:]
                assistant_response = self.generate_response(conversation, user_message)

                # Print opgehaalde context
                print("Opgehaalde context:")
                for t in self.search_knowledge(user_message, top_k=5):
                    print(f"- subject: {t['subject']}, predicate: {t['predicate']}, object: {t['object']}, (start: {t['start']}, end: {t['end']})")

                print(f"Assistant: {assistant_response}")
                self.save_message(role='assistant', content=assistant_response)
        else:
            print("Chatbot is ready! Type 'exit' to end the conversation.")
            while True:
                user_message = input("You: ")
                if user_message.lower().strip() in ['exit', 'quit']:
                    print("Chatbot: Goodbye!")
                    break
                self.save_message(role='user', content=user_message)
                conversation = self.load_json_data(self.messages_file)[-3:]
                assistant_response = self.generate_response(conversation, user_message)
                print(f"Assistant: {assistant_response}")
                self.save_message(role='assistant', content=assistant_response)

if __name__ == "__main__":
    chatbot = Chatbot()
    vragenlijst = [
        "Wat wordt er na de silencer op de luchtdruk control unit geplaatst? "
        ]
    chatbot.chat(questions=vragenlijst)


In [ ]:
import os
import json
import threading
from llama_cpp import Llama

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Qwen2.5-7B-Instruct-Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super().__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048)
            return cls._instance


class VIDAR:
    def __init__(self, output_file="vidar_memory_exp.json"):
        self.llm = LlamaSingleton().llm
        self.output_file = output_file
        self.memory = []
        self._load_memory()

    def _load_memory(self):
        if os.path.exists(self.output_file):
            with open(self.output_file, "r", encoding="utf-8") as f:
                self.memory = json.load(f)

    def _save_memory(self):
        with open(self.output_file, "w", encoding="utf-8") as f:
            json.dump(self.memory, f, indent=2)

    # def run_vidar_cycle(self, chunk_id, transcript, relations, hand_contacts):
    #     observations = f"Transcript: {transcript}\n" \
    #                    f"Linked object-action pairs: {', '.join(relations)}\n" \
    #                    f"Hand-object interactions: {', '.join(hand_contacts)}"

    #     messages = [
    #         {"role": "system", "content": (
    #             # "You are VIDAR, a reasoning entity. Analyze the following observations from a scene. "
    #             # "First, hypothesize what is happening, then reflect on what is uncertain. "
    #             # "Return a JSON with keys: hypothesis, reasoning, doubts (list), knowledge (triplets). "
    #             # "Each triplet has subject, predicate, object."
    #             "You are VIDAR, a reasoning entity. Analyze the following scene observations. " \
    #             "First, formulate a plausible hypothesis about what is occurring. " \
    #             "Then, reflect on uncertainties in the observations. " \
    #             "Respond in JSON format with the following keys: hypothesis (main interpretation), reasoning (explanation), doubts (list of unclear or ambiguous elements), knowledge (list of factual triplets). " \
    #             "Each triplet must be structured as: subject, predicate, object."
    #         )},
    #         {"role": "user", "content": observations}
    #     ]

    #     print(f"\nReasoning on chunk {chunk_id}...")
    #     print(observations)

    #     try:
    #         response = self.llm.create_chat_completion(
    #             messages=messages,
    #             temperature=0.6
    #         )
    #         content = response['choices'][0]['message']['content'].strip()

    #         # Verwijder eventuele Markdown-codeblokken
    #         if content.startswith("```json"):
    #             content = content[7:].strip()
    #         if content.endswith("```"):
    #             content = content[:-3].strip()

    #         # Escape line breaks binnen de "reasoning" waarde
    #         import re
    #         def escape_reasoning_block(match):
    #             inner = match.group(2).replace('\n', '\\n')
    #             return f'{match.group(1)}{inner}"'
    #         content = re.sub(
    #             r'("reasoning"\s*:\s*")([^"]*?)(?<!\\)"',
    #             escape_reasoning_block,
    #             content,
    #             flags=re.DOTALL
    #         )

    #         try:
    #             parsed = json.loads(content)
    #         except json.JSONDecodeError as json_err:
    #             print(f"JSON decode error in chunk {chunk_id}: {json_err}")
    #             print("Response content was:\n", content)
    #             return

    #         self.memory.append({
    #             "chunk_id": chunk_id,
    #             "observations": observations,
    #             "hypothesis": parsed.get("hypothesis", ""),
    #             "reasoning": parsed.get("reasoning", ""),
    #             "doubts": parsed.get("doubts", []),
    #             "knowledge": parsed.get("knowledge", [])
    #         })

    #         self._save_memory()

    #     except Exception as e:
    #         print(f"Failed to process chunk {chunk_id}: {e}")

    def run_vidar_cycle(self, chunk_id, transcript, relations, hand_contacts):
        observations = f"Transcript: {transcript}\n" \
                        f"Linked object-action pairs: {', '.join(relations)}\n" \
                        f"Hand-object interactions: {', '.join(hand_contacts)}" \

        messages = [
            {"role": "system", "content": (
                "You are VIDAR, a reasoning entity. Analyze the following observations from a scene. "
                "First, hypothesize what is happening, then reflect on what is uncertain. "
                "Return a JSON with keys: hypothesis, reasoning, doubts (list), knowledge (triplets). "
                "Each triplet has subject, predicate, object."
            )},
            {"role": "user", "content": observations}
        ]

        print(f"\nReasoning on chunk {chunk_id}...")
        print(observations)

        try:
            response = self.llm.create_chat_completion(
                messages=messages,
                response_format={
                    "type": "json",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "hypothesis": {"type": "string"},
                            "reasoning": {"type": "string"},
                            "doubts": {
                                "type": "array",
                                "items": {"type": "string"}
                            },
                            "knowledge": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "subject": {"type": "string"},
                                        "predicate": {"type": "string"},
                                        "object": {"type": "string"}
                                    },
                                    "required": ["subject", "predicate", "object"]
                                }
                            }
                        },
                        "required": ["hypothesis", "reasoning", "doubts", "knowledge"]
                    }
                },
                temperature=0.6
            )

            content = response['choices'][0]['message']['content'].strip()

            if content.startswith("```json"):
                content = content[7:].strip()
            if content.endswith("```"):
                content = content[:-3].strip()

            parsed = json.loads(content)

            self.memory.append({
                "chunk_id": chunk_id,
                "observations": observations,
                "hypothesis": parsed.get("hypothesis", ""),
                "reasoning": parsed.get("reasoning", ""),
                "doubts": parsed.get("doubts", []),
                "knowledge": parsed.get("knowledge", [])
            })
            self._save_memory()

        except (json.JSONDecodeError, KeyError) as e:
            print(f"JSON parsing error in chunk {chunk_id}: {e}")
            print("🔍 Model response:\n", response)
        except Exception as e:
            print(f"Failed to process chunk {chunk_id}: {e}")



def main():
    chunks_path = "processed_json/stabilizer pressure control_chunks.json"
    enrichment_path = "enriched_filtered_captions_to_chunks_mapping_handsv2.json"

    if not os.path.exists(chunks_path):
        print(f"Bestand '{chunks_path}' niet gevonden.")
        return

    with open(chunks_path, "r", encoding="utf-8") as f:
        chunks_data = json.load(f).get("chunks", [])

    enrichment_map = {}
    if os.path.exists(enrichment_path):
        with open(enrichment_path, "r", encoding="utf-8") as f:
            enriched = json.load(f)
        for item in enriched:
            chunk_id = item.get("linked_chunk_id")
            if chunk_id:
                # Bewaar alleen de eerste match per chunk_id
                if chunk_id not in enrichment_map:
                    enrichment_map[chunk_id] = item

    vidar = VIDAR()

    for chunk in chunks_data:
        start = chunk.get("start")
        end = chunk.get("end")
        transcript = chunk.get("text", "").strip()
        chunk_id = f"{start}_{end}"

        if not transcript:
            print(f"⏭ Skip: Lege transcript in chunk {chunk_id}")
            continue

        enrichment = enrichment_map.get(chunk_id, {})
        relations = [
            f"{l['class_name']} → {l['action_label']}"
            for l in enrichment.get("object_action_links", [])
        ]
        hand_contacts = sorted({
            h.get("class_name", "").strip()
            for h in enrichment.get("hand_object_links", [])
            if h.get("class_name", "").strip()
        })
        # hand_contacts = sorted({
        #     f"{h.get('class_name', '').strip()} ({h.get('contact_type')})"
        #     for h in enrichment.get("hand_object_links", [])
        #     if h.get("class_name") and h.get("contact_type")
        # })

        vidar.run_vidar_cycle(chunk_id, transcript, relations, hand_contacts)

    print("\nVIDAR reasoning compleet. Resultaten in 'vidar_memory.json'.")


if __name__ == "__main__":
    main()
